# Projection de la température pour 2100

In [1]:
# Modules généraux
import numpy as np
import math
# Modules de graphisme
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Select, Div
from bokeh.plotting import ColumnDataSource, figure, output_file, show

In [2]:
class Projection():

    def __init__(self,year_for_change=2020):

        # Paramètres
        self.timeStep = 5       # pas de temps [an]
        self.eqCO2 = 280        # concentration en CO2 à l'équilibre [ppm]
        self.initCO2 = 290      # concentration en CO2 initiale [ppm]
        self.CO2_exp = 0.0225   # taux de croissance [/an]
        self.CO2RampExp = 0.01  # taux de rabattement [/an]
        self.aerosol_Wm2_now = -0.75  # forçage radiatif des aérosols [W/m2]
        self.watts_m2_2x = 4
        self.climateSensitivity2x = 3  # sensibilité climatique pour un changement de T équivalent à un doublement de CO2 (soit: FR = 4 W/m2)
        self.climateSensitivityWm2 = self.climateSensitivity2x / self.watts_m2_2x  # changement de T par W/m2 de forçage
        self.TResponseTime = 20  # combinaison entre équilibrage rapide (océan superficiel) et équilibrage lent (océan profond)

       
        # Variables
        self.years = [1900]
       
        # Scénario "Business-as-usual"
        # CO2 atmosphérique
        self.bauCO2 = [self.initCO2]
        self.inoCO2 = [0]
        # Forçages radiatifs pour le scénario "business-as-usual"
        self.rfCO2 = [0]
        self.rfMask = [0]
        self.rfTot = [0]
        # Températures
        self.Teq = [0]
        self.TTrans = [0]
       
        # Scénario "World without us"
        # CO2 atmosphérique
        self.rampCO2 = [self.initCO2]
        # Forçages radiatifs pour le scénario "world without us"
        self.rfCO2Ramp = [0]
        self.rfMaskRamp = [0]
        self.rfTotRamp = [0]
        # Températures
        self.TeqRamp = [0]
        self.TTransRamp = [0]

       
        # Exécution de la routine "scenario" (paramètre: année de cessation des émissions)
        self.scenario(year_for_change)

       
    def scenario(self,year_for_change):

        # Scénario BUSINESS AS USUAL

        while self.years[-1] < 2100:
            # Création d'une liste d'années allant de 1900 à 2100 avec un pas de 1 an
            self.years.append(self.years[-1] + self.timeStep )
            # Calcul des concentrations de CO2
            self.bauCO2.append(self.eqCO2 + (self.bauCO2[-1] - self.eqCO2) * (1 + self.CO2_exp * self.timeStep))
            self.inoCO2.append( (self.bauCO2[-1] - self.bauCO2[-2]) / self.timeStep )
            # Calcul du forçage radiatif associé à la hausse de concentration de CO2
            self.rfCO2.append(self.watts_m2_2x * math.log( self.bauCO2[-1]/self.eqCO2 ) / math.log(2) )

        # Recherche de l'indice de l'année de cessation des émissions de CO2 dans la liste d'années
        iyrfc = self.years.index(year_for_change)
        # Calcul de la valeur de "B" tel que B*taux de croissance des émissions de CO2 actuel = FR
        aerosolCoeff = self.aerosol_Wm2_now / ( (self.bauCO2[iyrfc] - self.bauCO2 [iyrfc - 1]) / self.timeStep )

        for i in range(1, len(self.years)):
            # Calcul de l'effet masque (effet radiatif aérosols + GES à courte durée de vie)
            self.rfMask.append( max(self.inoCO2[i] * aerosolCoeff, self.aerosol_Wm2_now))
            # Calcul du forçage radiatif total : FR CO2 + FR masque
            self.rfTot.append( self.rfCO2[i] + self.rfMask[i] )
            # Calcul de T équilibre à partir de FR total (en supposant qu'il est gardé constant suffisamment longtemps pour atteindre l'équilibre)
            self.Teq.append( self.rfTot[i] * self.climateSensitivityWm2)
            # Calcul de T "transitoire" (T ne s'équilibre pas instantannément principalement à cause du rôle intégrateur des océans)
            self.TTrans.append( self.TTrans[-1] + (self.Teq[i] - self.TTrans[-1]) * self.timeStep / self.TResponseTime )

        # Scénario WORLD WITHOUT US
        for i in range(1, iyrfc):
            # Copie des variables du scénario "Business as usual" jusqu'au point où les émissions divergent
            # Variables copiées : concentrations en CO2, FR CO2, T équilibre et T transitoire
            self.rampCO2.append( self.bauCO2[i] )
            self.rfCO2Ramp.append( self.rfCO2[i] )
            self.rfMaskRamp.append( self.rfMask[i] )
            self.TTransRamp.append( self.TTrans[i] )
            self.TeqRamp.append( self.Teq[i] )
            self.rfTotRamp.append( self.rfTot[i] )

        # Calcul de la décroissance de concentration en CO2 et du FR associé
        # NB: l'effet masque n'est plus pris en compte -> court terme (déposition des aérosols et courte durée de vie des GES autres que CO2)
        # Le changement de T débute l'année de cessation des émissions de CO2
        for i in range(iyrfc, len(self.years)):
            self.rampCO2.append( self.rampCO2[-1] + (self.eqCO2*1.2 - self.rampCO2[-1]) * (self.CO2RampExp * self.timeStep) )
            self.rfCO2Ramp.append( self.watts_m2_2x * math.log(self.rampCO2[i]/self.eqCO2) / math.log(2) )
            self.rfMaskRamp.append( 0 )
            self.rfTotRamp.append( self.rfCO2Ramp[i] )
            self.TeqRamp.append( self.rfCO2Ramp[i] * self.climateSensitivityWm2 )
            self.TTransRamp.append( self.TTransRamp[-1] + (self.TeqRamp[i] - self.TTransRamp[-1]) * self.timeStep / self.TResponseTime )



In [3]:
p2020 = Projection(2020)
p2030 = Projection(2030)
p2040 = Projection(2040)
p2050 = Projection(2050)
p2060 = Projection(2060)
p2070 = Projection(2070)
p2080 = Projection(2080)
p2090 = Projection(2090)
p2100 = Projection(2100)

In [ ]:
source = ColumnDataSource(data =dict({
    
    'x': p2020.years, 'y': p2020.bauCO2, 'y1': p2020.rampCO2, 'y2': p2030.rampCO2, 'y3': p2040.rampCO2,'y4': p2050.rampCO2,
    'y5': p2060.rampCO2, 'y6': p2070.rampCO2, 'y7': p2080.rampCO2,'y8': p2090.rampCO2, 'y_select': p2020.rampCO2,
    
}))

In [4]:
# la figure et sa légende
plot1 = figure(x_range=(1900,2100), y_range=(250,1000), plot_width=300, plot_height=300)
plot1.yaxis.axis_label = 'Teneur atmosphérique en CO2 [ppm]'

# les données à tracer
plot1.dash('x', 'y', source=source, line_width=5, line_alpha=0.6, color='firebrick')
plot1.line('x', 'y_select', source=source, line_width=5, line_alpha=0.6, color='darkslategray')

# le sélecteur
select = Select(title="Zéro émissions nettes en ... ", value="2020", 
                options=["2020", "2030", "2040", "2050", "2060", "2070", "2080", "2090"], width=300)

# la fonction de sélection
callback = CustomJS(args=dict(source=source), 
                     code="""

    var f = cb_obj.value;

    var data = source.data;

    var y0 = data['y_select'];
    
    var y1 = data['y1'];
    var y2 = data['y2'];
    var y3 = data['y3'];
    var y4 = data['y4'];
    var y5 = data['y5'];
    var y6 = data['y6'];
    var y7 = data['y7'];
    var y8 = data['y8'];
    
    
    if (f=="2020"){
        data['y_select'] = y1;
    } else if (f=="2030"){
        data['y_select'] = y2;
    } else if (f=="2040"){
        data['y_select'] = y3;
    } else if (f=="2050"){
        data['y_select'] = y4;
    } else if (f=="2060"){
        data['y_select'] = y5;
    } else if (f=="2070"){
        data['y_select'] = y6;
    } else if (f=="2080"){
        data['y_select'] = y7;
    } else {
        data['y_select'] = y8;
    }
    
    source.change.emit();
""")
select.js_on_change("value", callback)

# le décor
plot1.xgrid.grid_line_color = "lightgray"
plot1.ygrid.grid_line_color = "lightgray"
plot1.outline_line_width = 5
plot1.outline_line_color = "darkgreen"
plot1.toolbar.autohide = True

In [ ]:
source2 = ColumnDataSource(data =dict({
    
    'x': p2020.years,'y': p2020.rfTot, 'y1': p2020.rfTotRamp, 'y2': p2030.rfTotRamp, 'y3': p2040.rfTotRamp, 'y4': p2050.rfTotRamp,
    'y5': p2060.rfTotRamp, 'y6': p2070.rfTotRamp, 'y7': p2080.rfTotRamp, 'y8': p2090.rfTotRamp, 'y_select': p2020.rfTotRamp,
    
}))

In [5]:
# la figure et sa légende
plot2 = figure(x_range=(1900,2100), y_range=(0,8), plot_width=300, plot_height=300)
plot2.yaxis.axis_label = 'Forçage radiatif en W/m²'

# les données à tracer
plot2.dash('x', 'y', source=source2, line_width=5, line_alpha=0.6, color='firebrick')
plot2.line('x', 'y_select', source=source2, line_width=5, line_alpha=0.6, color='lawngreen')

# le sélecteur
select2 = Select(title="Forçage radiatif associé au ...", 
                value="scénario 2020", 
                options=["scénario 2020", "scénario 2030", "scénario 2040", 
                         "scénario 2050", "scénario 2060", "scénario 2070",
                         "scénario 2080", "scénario 2090"],
                width=300)

# la fonction de sélection
callback = CustomJS(args=dict(source=source2), 
                     code="""

    var f = cb_obj.value;

    var data = source.data;

    var y0 = data['y_select'];
    
    var y1 = data['y1'];
    var y2 = data['y2'];
    var y3 = data['y3'];
    var y4 = data['y4'];
    var y5 = data['y5'];
    var y6 = data['y6'];
    var y7 = data['y7'];
    var y8 = data['y8'];
    
    
    if (f=="scénario 2020"){
        data['y_select'] = y1;
    } else if (f=="scénario 2030"){
        data['y_select'] = y2;
    } else if (f=="scénario 2040"){
        data['y_select'] = y3;
    } else if (f=="scénario 2050"){
        data['y_select'] = y4;
    } else if (f=="scénario 2060"){
        data['y_select'] = y5;
    } else if (f=="scénario 2070"){
        data['y_select'] = y6;
    } else if (f=="scénario 2080"){
        data['y_select'] = y7;
    } else {
        data['y_select'] = y8;
    }
    
    source.change.emit();
""")
select2.js_on_change("value", callback)

# le décor
plot2.xgrid.grid_line_color = "lightgray"
plot2.ygrid.grid_line_color = "lightgray"
plot2.outline_line_width = 5
plot2.outline_line_color = "darkgreen"
plot2.toolbar.autohide = True

In [ ]:
source3 = ColumnDataSource(data =dict({
    
    'x': p2020.years, 'y': p2020.TTrans, 'y1': p2020.TTransRamp, 'y2': p2030.TTransRamp, 'y3': p2040.TTransRamp, 
    'y4': p2050.TTransRamp, 'y5': p2060.TTransRamp, 'y6': p2070.TTransRamp, 'y7': p2080.TTransRamp,'y8': p2090.TTransRamp,
    'y_select': p2020.TTransRamp,
    
}))

In [6]:
# la figure et sa légende
plot3 = figure(x_range=(1900,2100), y_range=(0,4), plot_width=300, plot_height=300)
plot3.yaxis.axis_label = 'Température de surface en °C'

# les données à tracer
plot3.dash('x', 'y', source=source3, line_width=5, line_alpha=0.6, color='firebrick')
plot3.line('x', 'y_select', source=source3, line_width=5, line_alpha=0.6, color='yellow')

# le sélecteur
select3 = Select(title="Evolution de la température moyenne associée au ... ",
                value="scénario 2020", 
                options=["scénario 2020", "scénario 2030", "scénario 2040", 
                         "scénario 2050", "scénario 2060", "scénario 2070",
                         "scénario 2080", "scénario 2090"],
                width=300)

# la fonction de sélection
callback = CustomJS(args=dict(source=source3), 
                     code="""

    var f = cb_obj.value;

    var data = source.data;

    var y0 = data['y_select'];
    
    var y1 = data['y1'];
    var y2 = data['y2'];
    var y3 = data['y3'];
    var y4 = data['y4'];
    var y5 = data['y5'];
    var y6 = data['y6'];
    var y7 = data['y7'];
    var y8 = data['y8'];
    
    
    if (f=="scénario 2020"){
        data['y_select'] = y1;
    } else if (f=="scénario 2030"){
        data['y_select'] = y2;
    } else if (f=="scénario 2040"){
        data['y_select'] = y3;
    } else if (f=="scénario 2050"){
        data['y_select'] = y4;
    } else if (f=="scénario 2060"){
        data['y_select'] = y5;
    } else if (f=="scénario 2070"){
        data['y_select'] = y6;
    } else if (f=="scénario 2080"){
        data['y_select'] = y7;
    } else {
        data['y_select'] = y8;
    }
    
    source.change.emit();
""")
select3.js_on_change("value", callback)

# le bilan
plot3.xgrid.grid_line_color = "lightgray"
plot3.ygrid.grid_line_color = "lightgray"
plot3.outline_line_width = 5
plot3.outline_line_color = "darkgreen"
plot3.toolbar.autohide = True

In [7]:
div = Div(text=""" <i>Les courbes rouges correspondent à l'évolution des paramètres pour le scénario Business-As-Usual.</i>""", 
          width=940, height=20, background="peachpuff")

layout = column(row(column(select,plot1),column(select2,plot2),column(select3,plot3)),div)

output_file("monde_futur.html", title="monde_futur.py")

show(layout,notebook_handle=True)